In [38]:
from datetime import date,timedelta
import pandas as pd
import re
import numpy as np

In [39]:
def allSundays(year):
   d = date(year, 1, 1)
   d += timedelta(days = 6 - d.weekday())
   while d.year == year:
      yield d
      d += timedelta(days = 7)

In [40]:
allSundaysList = []
for d in allSundays(2017):
    allSundaysList.append(str(d).replace('-',''))

In [41]:
#constants or configuration settings
pd.set_option('display.float_format', lambda x: '%.8f' % x)
reCleanSplChars = re.compile(r'[a-zA-Z \t$%,?<>]*')

In [42]:
def GetDataAndProcess(weekendDate):
    #import pdb;pdb.set_trace()
    snapshotUrl = 'https://coinmarketcap.com/historical/{0}/'.format(weekendDate)
    snapshotCoinDf = pd.read_html(snapshotUrl)[0]
    snapshotCoinDf.rename(columns={'#':'RankNum'},inplace=True)
    
    snapshotCoinDf['PriceClean'] = snapshotCoinDf['Price'].str.replace(reCleanSplChars,'').str.replace(r'[0-9]*(-)[0-9]*','')
    snapshotCoinDf['Volume24hClean'] = snapshotCoinDf['Volume (24h)'].str.replace(reCleanSplChars,'')
    snapshotCoinDf['P7dClean'] = snapshotCoinDf['% 7d'].str.replace(reCleanSplChars,'')
    snapshotCoinDf['PriceClean'] = pd.to_numeric(snapshotCoinDf['PriceClean'])
    snapshotCoinDf['Volume24hClean'] = pd.to_numeric(snapshotCoinDf['Volume24hClean'])
    snapshotCoinDf['P7dClean'] = pd.to_numeric(snapshotCoinDf['P7dClean'])
    return(snapshotCoinDf)

In [43]:
def filterGroupNCalcData(cleanedDataFrame):
    tempDf = cleanedDataFrame[(cleanedDataFrame['PriceClean']>0.0001) & (cleanedDataFrame['PriceClean']<0.01)].copy()
    return(tempDf['P7dClean'].groupby(pd.cut(tempDf['Volume24hClean'],[100,1000,10000,tempDf.Volume24hClean.max()]
                                             ,labels=['100-1000','1000-10000','10000-MaxVol'])).mean())

In [44]:
masterListVar = []
for iDay in allSundaysList:    
    t = 'tsDataFrame_'+iDay
    s = 'summaryDf_'+iDay
    dictDf = {}
    dictDf[t] = GetDataAndProcess(iDay).copy()
    dictDf[s] = filterGroupNCalcData(dictDf[t]).copy()
    ##tsDataFrame_+iDay = GetDataAndProcess(iDay).copy()
    ##summaryDf_+iDay = filterGroupNCalcData(tsDataFrame_+iDay).copy()
    masterListVar.append({'forDate':iDay,'dataSet':dictDf[t],'summaryDataSet':dictDf[s]})

In [45]:
excelWriterStream = pd.ExcelWriter('CoinsSummaryAnalysis.xlsx',engine='xlsxwriter')
finalSummaryDf = pd.DataFrame()
for mlv in masterListVar:
    #print(mlv['forDate']);
    #print(mlv['summaryDataSet'])
    mlv['dataSet'].to_excel(excelWriterStream,sheet_name=mlv['forDate'])
    mlv['summaryDataSet'].rename(mlv['forDate'],inplace=True)
    finalSummaryDf[mlv['forDate']] = mlv['summaryDataSet']

finalSummaryDf.to_excel(excelWriterStream,sheet_name='Summary')
excelWriterStream.save()